## Queries

### 9.

In [24]:
from neo4j import GraphDatabase
import pandas as pd

def get_dungeon_gold(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(l:Loot)
        RETURN sum(l.gold) as gold

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        
    return data

get_dungeon_gold('Fenglass, Sepulcher of the Terrible Emperors')

,gold
0,1227.0


### 10.

In [26]:
def get_mean_monster_lvl(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(m:Monster)
        RETURN avg(m.level) as average_level

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        
    return data

get_mean_monster_lvl('Fenglass, Sepulcher of the Terrible Emperors')

,average_level
0,8.366972


### 11.

In [27]:
def get_mean_relationships_in_dungeon(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[r:IS_CONNECTED]-(:Room)
        RETURN node.room_id as id , count(r) as number_of_relationships
        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        avg = data['number_of_relationships'].mean()
        # make a groupby to count the number of relationships and get the mean

        
    return avg

get_mean_relationships_in_dungeon('Fenglass, Sepulcher of the Terrible Emperors')

4.31858407079646

### 12.

In [28]:
def get_max_monster_lvl(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(m:Monster)
        RETURN m.name as monster_name, max(m.level) as max_level, node.room_id as room_id, node.room_name as room_name

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        max_level_row = data.loc[data['max_level'].idxmax()]
        
    return max_level_row

get_max_monster_lvl('Fenglass, Sepulcher of the Terrible Emperors')

monster_name             fire giant
max_level                        50
room_id                         289
room_name       magnificent cellar 
Name: 24, dtype: object

### 13.

In [30]:
def get_exp_per_room(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(m:Monster)
        RETURN node.room_id as room_id, node.room_name as room_name, sum(m.exp) as total_exp

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        # sort by total_exp
        data = data.sort_values(by='total_exp', ascending=False)
        data.reset_index(drop=True, inplace=True)
        
    return data

get_exp_per_room('Fenglass, Sepulcher of the Terrible Emperors').head()

,room_id,room_name,total_exp
0,289,magnificent cellar,5000
1,287,chapel,4600
2,321,fancy vault of sumo wrestlers,4600
3,320,jolly washroom,4000
4,477,wine cellar,3600


### 14.

In [31]:
def most_exp_room(dungeon_name):
    room = get_exp_per_room(dungeon_name=dungeon_name).iloc[0][['room_id', 'room_name']]
    return room

most_exp_room('Fenglass, Sepulcher of the Terrible Emperors')

room_id                      289
room_name    magnificent cellar 
Name: 0, dtype: object

## Filtrado colaborativo

### Sin plugins

In [33]:
def get_monster_reccomendations(room_id):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (r:Room {room_id: $room_id})-[:CONTAINS]->(m:Monster)<-[:CONTAINS]-(r2:Room)-[:CONTAINS]->(m2:Monster)
        RETURN DISTINCT m2.name as monster_name, m2.level as monster_level, m2.exp as monster_exp
        '''
        result = session.run(query, room_id=room_id)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        
    return data

get_monster_reccomendations(473).head()

,monster_name,monster_level,monster_exp
0,magmin,1,100
1,azer,4,450
2,imp,2,200
3,succubus,11,1100
4,earth elemental,18,1800


### Usando GDS

In [35]:
from graphdatascience import GraphDataScience

def get_top5_monster_recommendations(room_id):
    gds = GraphDataScience("neo4j://localhost:7687", auth=("neo4j", "BDII2023"))
    
    query = """
    MATCH (r:Room {room_id: $id}) WITH r
    MATCH (m:Monster) WHERE not (r)-[:CONTAINS]->(m)
    RETURN m.name as monster, gds.alpha.linkprediction.preferentialAttachment(r, m, {relationshipQuery: "CONTAINS"}) AS score
    ORDER BY score DESC LIMIT 5
    """
    result = gds.run_cypher(query, {"id": room_id})
    return result



get_top5_monster_recommendations(473)

,monster,score
0,chuul,2190.0
1,driad,2016.0
2,sea hag,1992.0
3,wererat,1230.0
4,gargoyle,1218.0


In [36]:
from graphdatascience import GraphDataScience

def get_top5_monster_recommendations(room_id):
    gds = GraphDataScience("neo4j://localhost:7687", auth=("neo4j", "BDII2023"))
    node_query = """
        MATCH (b1:Monster)<-[:CONTAINS]-(u:Room)-[:CONTAINS]->(b2:Monster) 
        WITH b1, b2, count(distinct u) as liked 
        RETURN DISTINCT id(b1) as id
    """
    edge_query = """
        MATCH (b1:Monster)<-[:CONTAINS]-(u:Room)-[:CONTAINS]->(b2:Monster) 
        WITH b1, b2, count(distinct u) as liked 
        RETURN id(b1) as source, id(b2) as target, (liked) as weight
    """

    with gds.graph.project.cypher("coliked_beers_temp", node_query, edge_query) as g_temp:
        q_source = """
            MATCH (:Room {room_id: $id})-[:CONTAINS]->(b:Monster) RETURN collect(id(b)) as sources
        """
        sources = gds.run_cypher(q_source,params={"id": str(room_id)}).sources[0]
        result = gds.pageRank.stream(g_temp, sourceNodes=sources, relationshipWeightProperty="weight")
        result = result.query("score > 0")

    nodes = result.nodeId.to_list()
    q = """
    MATCH (:Room {room_id: $id})-[:CONTAINS]->(b:Monster) WITH collect(b) as sources
    MATCH (b:Monster) WHERE id(b) IN $nodes AND not(b in sources)
    RETURN id(b) AS nodeId, b.name AS monster
    """
    df = gds.run_cypher(q, params={"id":str(room_id),"nodes": nodes})

    res = result.join(df.set_index("nodeId"), on="nodeId").dropna().sort_values("score", ascending=False).head(10)
    res.reset_index(drop=True, inplace=True)
    return res

get_top5_monster_recommendations(473)

,nodeId,score,monster
0,11288,2.580650,azer
1,11315,2.207019,spined devil
2,11443,2.044392,duergar
3,11312,2.029916,imp
4,11358,1.930144,harpy
5,11359,1.876028,hippogriff
6,11399,1.875842,scarecrow
7,11461,1.856567,kuo-toa whip
8,11378,1.839657,merrow
9,11301,1.807188,quasit
